In [2]:
search_for = 130
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002316713333129883
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 749112551
type: [1, 1, 1, 1, 130] 130
cases of this type: 285610000
100000 33.43546390854774
200000 58.31772963425092
300000 54.44672157546203
400000 55.96174317227625
500000 55.01262643066616
600000 58.04414720381582
700000 55.6517261399316
800000 53.79946054685458
900000 55.854772210014595
1000000 53.801239462917636
1100000 40.20054119584986
1200000 33.076633677001745
1300000 53.17943635417602
1400000 55.071036261297444
1500000 52.4260031192248
1600000 55.04054159257431
1700000 57.200319735731824
1800000 53.8780428434204
1900000 55.53034894408297
2000000 54.2435020830724
2100000 56.04581400839337
2200000 41.23487207106817
2300000 27.276088330691522
2400000 55.19461180121908
2500000 52.055043947950296
2600000 53.97856399404521
2700000 53.07600660603368
2800000 55.22595215062484
2900000 53.61909889299364
3000000 50.813991524926

29700000 9.671915611824751
29800000 24.057295355781
29900000 22.225906654261244
30000000 25.707408600696375
30100000 29.96604284846932
30200000 25.9334566477534
30300000 31.29079098025289
30400000 35.37126794275443
30500000 51.15337938615695
30600000 23.325927254248647
30700000 54.08902245245778
30800000 54.09485645214295
30900000 45.52371121147102
31000000 39.299396051858714
31100000 35.49737439501408
31200000 48.599408057101236
31300000 52.131160247767326
31400000 45.02521770588833
31500000 49.64327082724926
31600000 44.1695862702692
31700000 45.39981966776272
31800000 46.10350923212939
31900000 18.491537613583525
32000000 49.21641648785922
32100000 41.39522006712021
32200000 46.44889084555172
32300000 45.07125826243456
32400000 46.3654761833891
32500000 50.46656930037121
32600000 45.649999827600986
32700000 47.181942249827934
32800000 29.698361163243515
32900000 45.591898956932475
33000000 49.92623595990128
33100000 46.39579110469913
33200000 45.349404076902
33300000 31.201657513156

59800000 29.82624489930874
59900000 45.854091432138254
60000000 51.70477440823486
60100000 49.993633474291116
60200000 52.563187563243744
60300000 46.28936698095091
60400000 47.54758730367908
60500000 24.70728566163046
60600000 49.97191384919787
60700000 50.17299246704715
60800000 47.45195300490921
60900000 53.016311155210424
61000000 48.89246083085457
61100000 25.991453394106358
61200000 52.281417002062724
61300000 52.90650547107513
61400000 59.28767842498586
61500000 51.452536854344075
61600000 56.5730936874244
61700000 52.12620554967256
61800000 52.77162961457621
61900000 55.33539778968822
62000000 35.219362862978244
62100000 37.40852907868364
62200000 48.73468367754025
62300000 48.552147027433236
62400000 49.46265247782343
62500000 46.101318707429456
62600000 44.50173100764155
62700000 25.827081097915755
62800000 47.627163370193074
62900000 48.90987213352462
63000000 46.93986219874088
63100000 50.47250383391872
63200000 49.88229841964911
63300000 21.699224805643365
63400000 54.7226

89900000 50.742821000053596
90000000 52.57503931198202
90100000 53.844835213359175
90200000 54.87998640239237
90300000 55.511419533403775
90400000 51.83610635143164
90500000 53.34023571863813
90600000 51.07190042604027
90700000 49.4949939706482
90800000 26.653278579361128
90900000 42.27387511299517
91000000 49.31436156240518
91100000 49.313121332000044
91200000 26.68942868483379
91300000 41.55928495413747
91400000 48.0627639905886
91500000 48.541289410058454
91600000 31.032579929904532
91700000 40.11368297637458
91800000 50.26415460031543
91900000 50.19814568843694
92000000 52.18117014213119
92100000 52.97147479783619
92200000 54.6465813062446
92300000 55.2298382463967
92400000 46.97605120867842
92500000 50.41517217827924
92600000 48.336945930074165
92700000 48.77190665636108
92800000 45.798098798898735
92900000 42.90327897991222
93000000 29.610278506037545
93100000 34.954046081382145
93200000 44.18716449723119
93300000 44.163990627056705
93400000 24.516682095376527
93500000 40.2874142

119300000 37.02410398817814
119400000 40.840678544606355
119500000 39.9796066356696
119600000 17.713782259227894
119700000 44.06617508131578
119800000 12.993444544805879
119900000 41.22152240240903
120000000 23.32479218498173
120100000 38.87360077290745
120200000 41.95369119588512
120300000 36.76638177755054
120400000 42.32287772083506
120500000 38.24608861228055
120600000 39.90052606508531
120700000 43.92575520398497
120800000 40.88802272939316
120900000 45.062950101484695
121000000 41.46658458417163
121100000 43.06764728855143
121200000 41.85485847300617
121300000 37.46835423814879
121400000 43.0728060706109
121500000 37.28726085672542
121600000 41.34862075153635
121700000 39.9600986586272
121800000 21.57203715589002
121900000 43.19019669665088
122000000 13.457840750932645
122100000 35.588159704636865
122200000 29.118304144649446
122300000 38.75052105355949
122400000 42.33268889372516
122500000 36.96728042075957
122600000 42.778776647665865
122700000 39.07041757951777
122800000 39.44

...

638700000 1.1810547476368758
638800000 0.7797286605279075
638900000 1.242320612879902
639000000 1.3439792597306734
639100000 1.6384309829979273
639200000 1.145326766955485
639300000 1.104007776254897
best so far: 0
type: [1, 1, 2, 13, 5] 130
cases of this type: 5492500
639400000 1.725495003976957
639500000 1.347627559901919
639600000 1.2860283499882923
639700000 1.040736455005141
639800000 1.3661521300513206
639900000 1.1069830804073226
640000000 0.9669958984310971
640100000 1.0008824363149702
640200000 1.7325668182218281
640300000 1.9461888813024137
640400000 1.0882129311028441
640500000 1.513343231074343
640600000 1.5725845657744175
640700000 1.27178233124004
640800000 1.2471696152573355
640900000 1.0789599073986031
641000000 0.8928129589416484
641100000 1.1969742957048408
641200000 1.7697867181063447
641300000 1.3949923948881804
641400000 1.0300515944970627
641500000 1.0124407357566627
641600000 1.411325708823055
641700000 1.4069175663595472
641800000 1.2465856826879111
641900000 0.

665500000 0.5185162881878522
665600000 0.34540564972533777
665700000 0.3447282168812447
665800000 0.34784932713471584
665900000 0.3522647411932468
666000000 0.35239976883850627
666100000 0.5257940347380665
666200000 0.38469735696829555
666300000 0.34255675750881703
666400000 0.6391479651878225
666500000 0.9453580477366945
666600000 1.0728738382578544
666700000 0.7687516234448175
666800000 1.1484775030907213
666900000 0.8339755992088306
667000000 1.1730142398742822
667100000 1.6755814828046263
667200000 1.0897359705500071
best so far: 0
type: [1, 1, 10, 13, 1] 130
cases of this type: 219700
667300000 0.49163642443613476
667400000 0.3093539458121777
best so far: 0
type: [1, 1, 13, 1, 10] 130
cases of this type: 1690000
667500000 0.8994583684900549
667600000 1.450408402247932
667700000 2.0339479932892024
667800000 2.4859629011601063
667900000 2.011969773316631
668000000 1.6016077861934601
668100000 0.45131242680898637
668200000 0.38379903110399183
668300000 0.3464810709020873
668400000 0.

691300000 1.4782444855321586
691400000 1.3636477735487582
691500000 1.4360085234333746
691600000 1.5707547934447421
691700000 1.3578616123143135
691800000 1.2131150752651234
691900000 2.063049547798403
692000000 1.8209715097300172
692100000 1.2823531125980236
692200000 1.1984072713768132
692300000 0.6211784176018973
692400000 1.5104019550196728
692500000 1.2821935645959788
692600000 1.3892440334412657
692700000 1.605158939072821
692800000 0.9648453490526484
692900000 0.6863248076911865
693000000 1.4005169853707027
693100000 1.7452555103565903
693200000 2.5766641507191417
693300000 1.7989393006766814
693400000 0.9536588708053297
693500000 0.9689208988115846
693600000 1.9948215303610992
693700000 1.760237153434495
693800000 1.7814352260898232
693900000 1.4348612872033637
694000000 0.7559047142795027
694100000 1.3265686545695603
694200000 2.2269650303519954
694300000 1.3886094014359747
694400000 1.7896172806340196
694500000 1.5276017786133216
694600000 0.9263020007941762
694700000 1.29821

718500000 0.23977000813755125
718600000 0.25271999679568474
best so far: 0
type: [1, 2, 5, 13, 1] 130
cases of this type: 109850
718700000 0.26509409665599665
best so far: 0
type: [1, 2, 13, 1, 5] 130
cases of this type: 211250
718800000 0.16091089263429445
718900000 0.1255283087680724
719000000 0.12018590746077897
best so far: 0
type: [1, 2, 13, 5, 1] 130
cases of this type: 42250
best so far: 0
type: [1, 2, 65, 1, 1] 130
cases of this type: 8450
best so far: 0
type: [1, 5, 1, 1, 26] 130
cases of this type: 2284880
719100000 0.27764081678925356
719200000 0.44032392854691876
719300000 0.3993930861544649
719400000 0.33964389403404
719500000 0.3547018934220506
719600000 0.3631718009234428
719700000 0.6973861308124648
719800000 0.700614548454343
719900000 0.5676602350263714
720000000 0.38937493715676463
720100000 0.3357479214048048
720200000 0.11735520626663568
720300000 0.11997932703576618
720400000 0.11968058197685083
720500000 0.46506481878060696
720600000 0.6512164639992548
720700000 

738700000 0.16532568375324608
738800000 0.21010873438936473
738900000 0.19887467719335292
739000000 0.17403207443501814
739100000 0.17174546140574018
739200000 0.21662294798362916
739300000 0.16159797435604797
739400000 0.19191883749092223
739500000 0.18147446239391699
739600000 0.28139848610195123
739700000 0.15891710795116423
739800000 0.26022317701053815
739900000 0.2047049664439877
740000000 0.20768891938292186
740100000 0.15594019379081925
740200000 0.17268433874709144
740300000 0.16501130986979803
740400000 0.14544207011854315
740500000 0.1718969992187434
740600000 0.17491829905756115
740700000 0.13902099050115083
740800000 0.13552233364222246
740900000 0.15320134902193147
741000000 0.19351563135717842
741100000 0.16500496307204832
741200000 0.14571914333223304
741300000 0.18983763494945233
741400000 0.18242168904926512
741500000 0.13888998199017644
741600000 0.13395003555890586
741700000 0.16294572703283763
741800000 0.13892527516654135
best so far: 0
type: [2, 1, 1, 5, 13] 130
